In [21]:
# !pip install chardet
# !pip install pyarrow
# !pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 374.3 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 980.5 kB/s eta 0:00:000:00:01
  Created wheel for kaggle: filename=kaggle-1.6.6-py3-none-any.whl size=111964 sha256=8b9eef07d7d834daf74665fd8e6584b8e558edbcef5f0693b8aba79b4e7dbe6f
  Stored in directory: /Users/oliviapc/Library/Caches/pip/wheels/46/aa/c3/b3e421522fb5acdd7c366a05c5fc80787615bdeed207e7f79b
Successfully built kaggle

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [4]:
import pandas as pd
import requests
from io import StringIO
import chardet

## Descàrrega API Compravenda

In [5]:
response = requests.get('https://opendata-ajuntament.barcelona.cat/data/es/dataset/est-mercat-immobiliari-compravenda-sup/resource/56b5a20f-078f-40f3-a637-bfa455d0ce1d/download')

In [6]:
if response.status_code == 200:
    # Detecta automáticamente la codificación del contenido
    encoding = chardet.detect(response.content)['encoding']
    
    # Decodifica el contenido con la codificación detectada
    contenido = response.content.decode(encoding)
    
    # Utilizamos StringIO para convertir el contenido en un archivo que pd.read_csv pueda leer
    csv_file = StringIO(contenido)
    
    # Utilizamos pd.read_csv para leer el contenido CSV
    df = pd.read_csv(csv_file)

else:
    print("La solicitud no fue exitosa. Código de estado:", response.status_code)


In [9]:
df.to_parquet('data_lake/compravenda_sup.parquet')

## Descàrrega API Renda Tributària per llar

In [10]:
response = requests.get('https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/44916a85-462d-4c1c-b3a5-98d9e161c299/download')

In [11]:
if response.status_code == 200:
    # Detecta automáticamente la codificación del contenido
    encoding = chardet.detect(response.content)['encoding']
    
    # Decodifica el contenido con la codificación detectada
    contenido = response.content.decode(encoding)
    
    # Utilizamos StringIO para convertir el contenido en un archivo que pd.read_csv pueda leer
    csv_file = StringIO(contenido)
    
    # Utilizamos pd.read_csv para leer el contenido CSV
    df = pd.read_csv(csv_file)

else:
    print("La solicitud no fue exitosa. Código de estado:", response.status_code)

In [12]:
df

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros
0,2021,1,Ciutat Vella,1,el Raval,1,28484
1,2021,1,Ciutat Vella,1,el Raval,2,23611
2,2021,1,Ciutat Vella,1,el Raval,3,26607
3,2021,1,Ciutat Vella,1,el Raval,4,28688
4,2021,1,Ciutat Vella,1,el Raval,5,23769
...,...,...,...,...,...,...,...
1063,2021,10,Sant Martí,73,la Verneda i la Pau,143,34553
1064,2021,10,Sant Martí,65,el Clot,234,38965
1065,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,235,29774
1066,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,236,32925


In [13]:
df.to_parquet('data_lake/renda.parquet')

## Descàrrega API preu del lloguer (Kaggle)

Abans s'ha de:
1. Crear un compte a Kaggle
2. Anar a Settings i fer click a Create New API Token
3. Guardar el fitxer kaggle.json que acabes de descarregar a la carpeta ~/.kaggle

In [28]:
#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context

In [30]:
!kaggle datasets download -d marshuu/rent-price-in-barcelona-2014-2022

/Users/oliviapc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
  0%|                                               | 0.00/33.6k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 33.6k/33.6k [00:00<00:00, 567kB/s]


In [31]:
!unzip rent-price-in-barcelona-2014-2022.zip

Archive:  rent-price-in-barcelona-2014-2022.zip
  inflating: Barcelona_rent_price.csv  


In [32]:
pd.read_csv('Barcelona_rent_price.csv').to_parquet('data_lake/rent_price.parquet')

In [33]:
!rm rent-price-in-barcelona-2014-2022.zip Barcelona_rent_price.csv